In [39]:
cd

/Users/dclbenedicto


/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/opt/homebrew/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/IPython/core/magics/osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Loading data

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import re
import numpy as np
import pandas as pd
import hashlib
import seaborn as sns
import matplotlib.pyplot as plt
import honors.helper as h

In [42]:
#load all data
responses_101 = pd.read_csv('honors/Copy of Data 101 Fa23 Extensions Backend - Form Responses.csv')
roster_101 = pd.read_csv('honors/Copy of Data 101 Fa23 Extensions Backend - Roster.csv')
evaluation_101 = pd.read_csv('honors/Internal Final Survey – Data 101, Fall 2023 (Responses) - Form Responses 1.csv')

responses_8 = pd.read_csv('honors/[FA23] Data 8 Extensions Data  - Extension Form Responses.csv')
roster_8 = pd.read_csv('honors/[FA23] Data 8 Extensions Data  - Extension Roster.csv')
evaluation_8 = pd.read_csv('honors/[FA23] Data 8 Extensions Data  - End of Semester Evaluation.csv')

responses_100 = pd.read_csv('honors/responses_100.csv')
roster_100 = pd.read_csv('honors/roster_100.csv')
evaluation_100 = pd.read_csv('honors/evaluation_100.csv')

In [43]:
list_of_tables = [responses_101, responses_8, responses_100,
                  roster_101, roster_8, roster_100,
                  evaluation_101, evaluation_8, evaluation_100]
                  
# responses_8, roster_8, evaluation_8, responses_100, roster_100, evaluation_100]

# Data Cleaning

In [44]:
#remove NaNs
responses_101.dropna(subset = ['Email Address'], inplace=True)
roster_101.dropna(subset = ['email'], inplace=True)

### Data Deidentification

In [45]:
#apply to all rows for all df w/ email
responses_101['rid'] = responses_101['Email Address'].apply(h.hash_email)
roster_101['rid'] = roster_101['email'].apply(h.hash_email)

### Fixing columns + column names

In [46]:
roster_100.columns

Index(['Unnamed: 0', 'RID', 'is_dsp', 'hw1', 'hw2a', 'hw2b', 'hw3', 'hw4',
       'hw5', 'hw6', 'hw7', 'hw8', 'lab1', 'lab2a', 'lab2b', 'lab3', 'lab4',
       'lab5', 'lab6', 'lab7', 'lab8', 'lab9', 'lab10', 'lab11', 'lab12',
       'lab13', 'proja1', 'proja2', 'projb1', 'projb2'],
      dtype='object')

In [47]:
responses_101.columns

Index(['Timestamp', 'Email Address', 'Name', 'Student ID Number',
       'Are you submitting this form to request support for a specific lab, homework, or project?',
       'Are you a registered DSP student with assignment accommodations?',
       'What is the reason? What can we do to support you?',
       'When are you free to talk to a TA?',
       'For what assignment(s) are you requesting additional support?',
       'By how many days would you like the assignment extended?',
       'If you have documentation to support your request, please include it here',
       'What's going on? What kind of guidance would you like to discuss?',
       'Rerun', 'rid'],
      dtype='object')

In [48]:
# cleaning for necessary columns
#responses: RID, timestamp, reason (NaN if dsp), which assignment, requested days
responses_101 = responses_101[['rid', 'Timestamp', 'What is the reason? What can we do to support you?',
"What's going on? What kind of guidance would you like to discuss?", 'By how many days would you like the assignment extended?']]
responses_8 = responses_8[['RID', 'Timestamp', 'How many days would you like an extension for?', 'Why do you need the extension?']]
responses_100 = responses_100.iloc[:, 1:]

In [49]:
# cleaning for necessary columns
#roster: rid, is_dsp, granted days of all assignments
roster_101 = roster_101[['rid', 'is_dsp', 'proj1', 'multi1', 'proj2', 'multi2','multi3','proj3', 'multi4', 'multi5', 'proj4']]
roster_8 = roster_8[['RID', 'lab1', 'hw1', 'lab2', 'hw2', 'lab3', 'hw3','lab4', 'hw4', 'hw5',
                     'lab5', 'proj1', 'hw6', 'lab6', 'hw7', 'lab7', 'hw8', 'lab8', 'hw9', 'hw10',
                     'lab9', 'proj2', 'hw11', 'lab10', 'hw12', 'proj3']]
roster_100 = roster_100[['RID', 'is_dsp', 'lab1', 'hw1', 'lab2a', 'hw2a', 'lab2b', 'hw2b', 'lab3', 'hw3',
                        'lab4', 'hw4', 'lab5', 'hw5', 'lab6', 'hw6', 'lab7', 'proja1', 'lab8', 'proja2',
                         'lab9', 'hw7', 'lab10', 'hw8', 'lab11', 'projb1', 'projb2', 'lab12', 'lab13']]

In [50]:
# cleaning for necessary columns
#evaluation questions
evaluation_101 = evaluation_101.iloc[:, 59:]
evaluation_101.rename(columns={evaluation_101.columns[0]: 'participate'}, inplace=True)
evaluation_101.drop(evaluation_101[evaluation_101['participate'] == 'No'].index, inplace=True)
evaluation_101.drop(evaluation_101.columns[0], axis=1, inplace=True)

evaluation_100 = evaluation_100.iloc[:, 1:]

In [51]:
h.merge_reasons(responses_101)
h.merge_reasons(responses_100)

/Users/dclbenedicto/honors/helper.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reason'] = df[df.columns[2]].fillna(df[df.columns[3]])
/Users/dclbenedicto/honors/helper.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([df.columns[2], df.columns[3]], axis=1, inplace=True)


In [52]:
#renaming responses
for df in [responses_8, responses_100, responses_101]:
    for i in np.arange(0,len(responses_8.columns)):
        new_cols = ['rid', 'timestamp', 'requested_days', 'reason']
        h.rename_columns(df, i, new_cols[i])

#renaming evaluations
for i in np.arange(0,len(evaluation_8.columns)):
    new_cols = ['rid', 'usage', 'comfort', 'comfort_reason', 'learning_benefit', 'experience_benefit', 'reason_benefit', 'feedback']
    h.rename_columns(evaluation_8, i, new_cols[i])

for i in np.arange(0,len(evaluation_100.columns)):
    new_cols = ['usage', 'comfort', 'comfort_reason', 'learning_benefit', 'experience_benefit', 'reason_benefit', 'feedback']
    h.rename_columns(evaluation_100, i, new_cols[i])

for i in np.arange(0,len(evaluation_101.columns)):
    new_cols = ['usage', 'comfort', 'comfort_reason', 'learning_benefit', 'experience_benefit', 'reason_benefit', 'feedback']
    h.rename_columns(evaluation_101, i, new_cols[i])

h.rename_columns(roster_8, 0, 'rid')
h.rename_columns(roster_100, 0, 'rid')

/Users/dclbenedicto/honors/helper.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={df.columns[i]: new_name}, inplace=True)


In [53]:
#for all text, replace w/ 'redacted' if identifiable information is there
students_101 = pd.read_csv('honors/Data_101_Fall_2023_roster (1).csv')
sensitive_info_101 = h.compile_sensitive_info(students_101)
h.redact_info(responses_101, 'reason', sensitive_info_101)

students_8 = pd.read_csv('honors/Data_8_Fall_2023_roster.csv')
students_8[['First Name', 'Last Name']] = students_8['Name'].str.split(n=1, expand=True)
sensitive_info_8 = h.compile_sensitive_info(students_8)
h.redact_info(responses_8, 'reason', sensitive_info_8)

students_100 = pd.read_csv('honors/Data_100_Fall_2023_roster (4).csv')
sensitive_info_100 = h.compile_sensitive_info(students_100)
h.redact_info(responses_100, 'reason', sensitive_info_100)

/Users/dclbenedicto/honors/helper.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace(regex_pattern, '[redacted]', regex=True)


In [54]:
list_of_tables = [responses_101, responses_8, responses_100,
                  roster_101, roster_8, roster_100,
                  evaluation_101, evaluation_8, evaluation_100]

for i in list_of_tables:
    print(i.columns)

Index(['rid', 'timestamp', 'requested_days', 'reason'], dtype='object')
Index(['rid', 'timestamp', 'requested_days', 'reason'], dtype='object')
Index(['rid', 'timestamp', 'requested_days', 'reason'], dtype='object')
Index(['rid', 'is_dsp', 'proj1', 'multi1', 'proj2', 'multi2', 'multi3',
       'proj3', 'multi4', 'multi5', 'proj4'],
      dtype='object')
Index(['rid', 'lab1', 'hw1', 'lab2', 'hw2', 'lab3', 'hw3', 'lab4', 'hw4',
       'hw5', 'lab5', 'proj1', 'hw6', 'lab6', 'hw7', 'lab7', 'hw8', 'lab8',
       'hw9', 'hw10', 'lab9', 'proj2', 'hw11', 'lab10', 'hw12', 'proj3'],
      dtype='object')
Index(['rid', 'is_dsp', 'lab1', 'hw1', 'lab2a', 'hw2a', 'lab2b', 'hw2b',
       'lab3', 'hw3', 'lab4', 'hw4', 'lab5', 'hw5', 'lab6', 'hw6', 'lab7',
       'proja1', 'lab8', 'proja2', 'lab9', 'hw7', 'lab10', 'hw8', 'lab11',
       'projb1', 'projb2', 'lab12', 'lab13'],
      dtype='object')
Index(['usage', 'comfort', 'comfort_reason', 'learning_benefit',
       'experience_benefit', 'reason_benef

In [57]:
#exporting the data
responses_101.to_csv('honors/responses_101.csv')
roster_101.to_csv('honors/roster_101.csv')
evaluation_101.to_csv('honors/evaluation_101.csv')

responses_8.to_csv('honors/responses_8.csv')
roster_8.to_csv('honors/roster_8.csv')
evaluation_8.to_csv('honors/evaluation_8.csv')

responses_100.to_csv('honors/responses_100.csv')
roster_100.to_csv('honors/roster_100.csv')
evaluation_100.to_csv('honors/evaluation_100.csv')